In [1]:
!pip3 -q install pylibjpeg pylibjpeg-libjpeg pydicom python-gdcm

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

import pydicom as dicom
import cv2 as cv
from scipy.sparse import load_npz

from tqdm import tqdm
import os
import json

from warnings import filterwarnings
filterwarnings('ignore')

from multiprocessing.pool import ThreadPool

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
dicom_path = Path('/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images/')
segm_path = Path('/kaggle/input/rsna-csfd-masks')

In [4]:
frac_df = pd.read_csv('/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train.csv')
frac_df.shape

(2019, 9)

In [5]:
# ЗАГЛУШКА ТОЛЬКО ДЛЯ СУЩЕСТВУЮЩИХ МАСОК

presented_masks = [mask_name[:-4] for mask_name in os.listdir(str(segm_path)) if '.npz' in mask_name]
frac_df = frac_df[frac_df.StudyInstanceUID.isin(presented_masks)].reset_index(drop=True)
frac_df.shape

(2015, 9)

In [6]:
with open(segm_path / 'metadata.json', 'r') as json_file:
    frac_df_metadata = json.load(json_file)

In [7]:
df = {'StudyInstanceUID' : [],
      'Vertebra' : [],
      'State' : [],
      'Slices' : []}

for _, record in frac_df.iterrows():
    for ind, (_, state) in enumerate(record[2:].items()):
        try:
            df['Slices'].append(','.join([str(slc) for slc in frac_df_metadata[record[0]][str(ind+1)]]))
            df['StudyInstanceUID'].append(record[0])
            df['Vertebra'].append(ind+1)
            df['State'].append(state)
        except Exception as e:
            print(e)
            continue
    
df = pd.DataFrame(df)
print(df.shape)
df.head()

'4'
'4'
'1'
'2'
'3'
'4'
'5'
'6'
'7'
'4'
(14095, 4)


,StudyInstanceUID,Vertebra,State,Slices
0,1.2.826.0.1.3680043.6200,1,1,"44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,5..."
1,1.2.826.0.1.3680043.6200,2,1,"49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,6..."
2,1.2.826.0.1.3680043.6200,3,0,"83,84,85,86,87,88,89,90,93,94,95,96,97,102,103"
3,1.2.826.0.1.3680043.6200,4,0,"84,85,87,88,90,91,92,93,97,98,99,100,101,102,1..."
4,1.2.826.0.1.3680043.6200,5,0,"91,92,99,100,102,103,104,105,106,107,108,109,1..."


In [8]:
df = df[df.Slices.apply(lambda x: len(x.split(',')) > 15)]
print(df.shape)

(14064, 4)


In [9]:
# Statistic about number of slices for each vertebra
pd.DataFrame([len(df.iloc[i][3].split(',')) for i in range(df.shape[0])]).describe()

,0
count,14064.000000
mean,55.820108
std,23.756604
min,16.000000
25%,39.000000
50%,50.000000
75%,67.000000
max,211.000000


In [10]:
os.mkdir('/kaggle/working/cropped_spines')

In [11]:
save_path = Path('/kaggle/working/cropped_spines')

In [12]:
df.shape

(14064, 4)

In [13]:
from tqdm.contrib.concurrent import process_map

In [14]:
# df = df.iloc[:100, :]

In [15]:
import sys
import numpy as np
import numpy.lib.format
import struct

def save(file, array):
    magic_string=b"\x93NUMPY\x01\x00v\x00"
    header=bytes(("{'descr': '"+array.dtype.descr[0][1]+"', 'fortran_order': False, 'shape': "+str(array.shape)+", }").ljust(127-len(magic_string))+"\n",'utf-8')
    if type(file) == str:
        file=open(file,"wb")
    file.write(magic_string)
    file.write(header)
    file.write(array.tobytes())
    
def load(file):
    if type(file) == str:
        file=open(file,"rb")
    header = file.read(128)
    if not header:
        return None
    descr = str(header[19:25], 'utf-8').replace("'","").replace(" ","")
    shape = tuple(int(num) for num in str(header[60:120], 'utf-8').replace(', }', '').replace('(', '').replace(')', '').split(','))
    datasize = numpy.lib.format.descr_to_dtype(descr).itemsize
    for dimension in shape:
        datasize *= dimension
    return np.ndarray(shape, dtype=descr, buffer=file.read(datasize))

def extract_proportional_elements(arr, T_size = 32):
    length_of_array = len(arr)

    if length_of_array == T_size:
        return arr
    elif length_of_array > T_size:
        step_size = length_of_array / T_size
        current_index = 0
        result_list = []
        for i in range(T_size):
            next_index = int(current_index + step_size)
            if next_index >= length_of_array:
                next_index = length_of_array - 1
            result_list.append(arr[next_index])
            current_index = next_index
        return result_list
    else:
        result_list = []
        num_copies = T_size // length_of_array
        for i in range(num_copies):
            result_list.extend(arr)

        remainder = T_size - len(result_list)

        if remainder != 0:
            step_size = length_of_array / remainder
            current_index = 0
            for i in range(remainder):
                next_index = current_index + step_size
                if next_index >= length_of_array:
                    next_index = length_of_array - 1
                result_list.append(arr[int(next_index)])
                current_index = next_index
        return sorted(result_list)

def compute_bounding_rect(mask):
    # compute rects
    bRects = [cv.boundingRect(mask[:, :, i]) for i in range(5, 32, 5)]
    # compute centers
    bRects = [(rect[0] + int(rect[2] / 2), rect[1] + int(rect[3] / 2)) for rect in bRects]
    # compute mean
    x, y = np.array(bRects).mean(axis=0).astype(int)
    # calculate compensation
    x_compensation = max(-(x - 112), 0) + min(-(x + 112 - 512), 0)
    y_compensation = max(-(y - 112), 0) + min(-(y + 112 - 512), 0)
    # apply compensation and use 
    x, y = x + x_compensation, y + y_compensation
    x1, y1, x2, y2 = (
        x - 112, y - 112, x + 112, y + 112
    )
    return (x1, y1, x2, y2)

def load_dicom(path):
    # Source: https://www.kaggle.com/code/vslaykovsky/pytorch-effnetv2-vertebrae-detection-acc-0-95
    try:
        img=dicom.dcmread(path)
    except:
        print('Error at load_dicom')
        return np.zeros((512, 512))
    img.PhotometricInterpretation = 'YBR_FULL'
    data = img.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data=(data * 255).astype(np.uint8)
    return data

def main(record):
    _, record = record
    uid = record[0]
    vertebra = record[1]
    state = record[2]
    slices = extract_proportional_elements([int(slc) for slc in record[3].split(',')])
    
    mask = load_npz(segm_path / (uid + '.npz'))[:, [slice_number - 1 for slice_number in slices]].toarray().reshape((512, 512, -1)).astype(np.uint8)
    mask = (mask == vertebra).astype(np.uint8)
    
    bRects = compute_bounding_rect(mask)
    
    imgs = np.zeros((512, 512, len(slices)))
    for i, slc in enumerate(slices):
        imgs[:, :, i] = load_dicom(dicom_path / uid / (str(slc) + '.dcm'))
    #imgs_masks = np.transpose(np.stack([imgs, mask]), axes=(0, 3, 1, 2))[:, :, bRects[1]:bRects[3], bRects[0]:bRects[2]].astype(np.uint8)
    imgs = np.transpose(imgs, axes=(2, 0, 1))[:, bRects[1]:bRects[3], bRects[0]:bRects[2]].astype(np.uint8)
    masks = np.packbits(np.transpose(mask, axes=(2, 0, 1))[:, bRects[1]:bRects[3], bRects[0]:bRects[2]].astype(bool), axis=None)
    
    np.savez_compressed(str(save_path / f'{uid}_{vertebra}_{state}'), imgs = imgs, masks=masks)
    
    #save(str(save_path / f'{uid}_{vertebra}_{state}_imgs.npy'), imgs)
    #save(str(save_path / f'{uid}_{vertebra}_{state}_masks.npy'), masks)

with ThreadPool(os.cpu_count()) as pool:
    max_ = df.shape[0]
    with tqdm(total=max_) as pbar:
        for _ in pool.imap_unordered(main, df.iterrows()):
            pbar.update()
    #tqdm(pool.map(main, df.iterrows()), total = df.shape[0])

#for _, record in tqdm(df.iterrows(), total=df.shape[0]):

 52%|█████▏    | 7272/14064 [50:33<1:37:18,  1.16it/s]

Error at load_dicom


100%|██████████| 14064/14064 [1:36:44<00:00,  2.42it/s]


In [16]:
# import zipfile
    
# def zipdir(path, ziph):
#     for root, dirs, files in os.walk(path):
#         for file in files:
#             ziph.write(os.path.join(root, file), 
#                        os.path.relpath(os.path.join(root, file), 
#                                        os.path.join(path, '..')))

# with zipfile.ZipFile('/kaggle/working/cropped_spines.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
#     zipdir('/kaggle/working/cropped_spines', zipf)

In [17]:
# import shutil
# shutil.rmtree(save_path)